reference :

https://www.kaggle.com/code/ammarnassanalhajali/uwmgi-unet-keras-inference/notebook

In [ ]:
! ls ../input/pycocotools

In [ ]:
!pip install -q pycocotools --no-index --find-links=file:///kaggle/input/pycocotools/ 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pycocotools
from pycocotools import mask
import json
import numpy as np
import pycocotools.mask as mask_util
from skimage import measure
import os
from tqdm import tqdm
from tqdm.notebook import tqdm
import cv2
import random
from itertools import groupby
import itertools
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from glob import glob
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib as mpl
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import *
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import load_model

In [ ]:
# creating a dataframe
DEBUG = True
current_path = "../input/uw-madison-gi-tract-image-segmentation/"
if DEBUG:
    path = "../input/uw-madison-gi-tract-image-segmentation/train/"
    file_name = "train.csv"
else:
    path = "../input/uw-madison-gi-tract-image-segmentation/test/"
    file_name = "sample_submission.csv"

print('file target', file_name)    
print('path', path)  

In [ ]:
dataset_path = os.path.abspath(path) 
csv_path = os.path.abspath(current_path+file_name)
df = pd.read_csv(csv_path)

def fetch_file_from_id(root_dir, case_id):
    case_folder = case_id.split("_")[0]
    day_folder = "_".join(case_id.split("_")[:2])
    file_starter = "_".join(case_id.split("_")[2:])
    # fetching folder paths
    folder = os.path.join(root_dir, case_folder, day_folder, "scans")
    # fetching filenames with similar pattern
    file = glob(f"{folder}/{file_starter}*")[0]
    # returning the first file, though it will always hold one file.
    return file

# df["segmentation"] = df["segmentation"].astype("str")
df["case_id"] = df["id"].apply(lambda x: x.split("_")[0][4:])
df["day_id"] = df["id"].apply(lambda x: x.split("_")[1][3:])
df["slice_id"] = df["id"].apply(lambda x: x.split("_")[-1])
df["path"] = df["id"].apply(lambda x: fetch_file_from_id(path, x))
df['path']=df.path.astype('str')
df['case_id']=df.case_id.astype('str')
df['slice_id']=df.slice_id.astype('str')
df["filename"] = df["path"].apply(lambda x: x.split('/')[-1])
df['unique_filename']  = df.apply(lambda row: str(row.case_id) +'_'+ str(row.day_id)+'_'+str(row.filename), axis=1)
df["height"] = df["path"].apply(lambda x: os.path.split(x)[-1].split("_")[2]).astype("int")
df["width"] = df["path"].apply(lambda x: os.path.split(x)[-1].split("_")[3]).astype("int")
df['size'] =df['height'].astype(str) +'x'+ df['width'].astype(str)
df.sample(5)

In [ ]:
df.shape

In [ ]:
import numpy as np
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape) # Needed to align to RLE direction


# From https://newbedev.com/encode-numpy-array-using-uncompressed-rle-for-coco-dataset
def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(itertools.groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle


# thanks to mohandass
# https://www.kaggle.com/code/mohanrobotics/cocoformat-dataset-creation-instance-segmentation
# i made some change to the id , the pycotools need numeric array id and not a string to work correctly
def create_coco_format_json(data_frame, classes, filepaths):
    images = []
    annotations = []
    categories = []
    count = 0
    
    # Additing categories
    for idx, class_ in enumerate(classes):
        categories.append(
            { 
                "id": idx,
                "name": class_
            }
        )
    i = 0
    for filepath in tqdm(filepaths):
        i += 1   
        image_id = i     
        file_id = ('_'.join((filepath.split("/")[-3] + "_" + filepath.split("/")[-1]).split("_")[:-4]))
        height_slice = int(filepath.split("/")[-1].split("_")[3])
        width_slice = int(filepath.split("/")[-1].split("_")[2])
        ids = data_frame.index[data_frame['id'] == file_id].tolist()
        file_name = '/'.join(filepath.split("/")[3:])
        if (len(ids) > 0):
            # Adding images which has annotations
            images.append(
                {
                    "id": image_id,
                    "file_id":file_id,#add id of the dataframe
                    "width":width_slice,
                    "height":height_slice,
                    "file_name": filepath
                }
            )

            for idx in ids:
                if(data_frame.iloc[idx]['segmentation'] != np.nan):
                
                    val = data_frame.iloc[idx]['segmentation']
                    if(pd.notnull(val)):
                        mk = rle_decode(val, (height_slice, width_slice))
                        ys, xs = np.where(mk)
                        x1, x2 = min(xs), max(xs)
                        y1, y2 = min(ys), max(ys)              
                        contours,hierarchy = cv2.findContours(mk,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
                        for id_, contour in enumerate(contours):
                            mask_image = np.zeros((mk.shape[0], mk.shape[1], 3),  np.uint8)
                            cv2.drawContours(mask_image, [contour], -1, (255,255,255), thickness=cv2.FILLED)
                            mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
                            mask_image_bool = np.array(mask_image, dtype=bool).astype(np.uint8)
                            ys, xs = np.where(mask_image_bool)
                            x1, x2 = min(xs), max(xs)
                            y1, y2 = min(ys), max(ys)
                            enc =binary_mask_to_rle(mask_image_bool)
                            seg = {
                                'segmentation':enc, 
                                'bbox': [int(x1), int(y1), int(x2-x1+1), int(y2-y1+1)],
                                'area': int(np.sum(mask_image_bool)),
                                'image_id':image_id, 
                                'category_id':classes.index(data_frame.iloc[idx]['class']), 
                                'iscrowd':0, 
                                'id': count
                            }
                            annotations.append(seg)
                            count +=1        
    # creating the dataset
    dataset_coco_format = {
        "categories": categories,
        "images": images,
        "annotations": annotations,
    }
    
    return dataset_coco_format

In [ ]:
classes = ['small_bowel', 'large_bowel', 'stomach']

test_json = create_coco_format_json(df, classes, df['path'].tolist())

In [ ]:
test_path_json = './train_all_json.json'

In [ ]:
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()
    
with open(test_path_json, 'w', encoding='utf-8') as f:
    json.dump(test_json, f, ensure_ascii=True, indent=4, default=np_encoder)

In [ ]:
from keras import backend as K
from keras.losses import binary_crossentropy
import tensorflow as tf

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + 0.5 * dice_loss(tf.cast(y_true, tf.float32), y_pred)

In [ ]:
class FixedDropout(keras.layers.Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape

        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)

custom_objects = custom_objects={
    'FixedDropout': FixedDropout,
    'dice_coef': dice_coef,
    'iou_coef': iou_coef,
    'bce_dice_loss': bce_dice_loss  
}

model = load_model('../input/keras-coco-training/model.h5', custom_objects=custom_objects)

In [ ]:
coco = COCO(test_path_json)
catIDs = coco.getCatIds()
imgIds = coco.getImgIds()
imgDict = coco.loadImgs(imgIds)
print(len(imgIds) , len(catIDs))

In [ ]:
def filterDataset( classes=None, json_file=None):    
    # initialize COCO api for instance annotations
    annFile = json_file
    coco = COCO(annFile)
    
    images = []
    if classes!=None:
        # iterate for each individual class in the list
        for className in classes:
            # get all images containing given categories
            catIds = coco.getCatIds(catNms=className)
            imgIds = coco.getImgIds(catIds=catIds)
            images += coco.loadImgs(imgIds)
    
    else:
        imgIds = coco.getImgIds()
        images = coco.loadImgs(imgIds)
    
    # Now, filter out the repeated images
    unique_images = []
    for i in range(len(images)):
        if images[i] not in unique_images:
            unique_images.append(images[i])
            
    random.shuffle(unique_images)
    dataset_size = len(unique_images)
    
    return unique_images, dataset_size, coco

In [ ]:
# use image size 128 for better results 
image_size = 128 
epochs = 10
batch_size = 8
input_image_size = (128,128)
classes = ['small_bowel', 'large_bowel', 'stomach']

In [ ]:
images_test, dataset_size_test, coco_val = filterDataset( classes,  test_path_json)

In [ ]:
class DataGeneratorFromCocoJson(tf.keras.utils.Sequence):
  # function getting info dataset from json coco
  # Batch size
  # subset train or test for annotations
  # image_list to develop...
  # classes classe wanted
  # input image size tuple (X,X)
  # annFile path to annoted coco json file file
    def __init__(self, batch_size=batch_size, subset="train", image_list=[], classes=[], input_image_size=(128, 128), annFile='', shuffle=False):

        super().__init__()
        self.subset = subset
        self.batch_size = batch_size
        self.indexes = np.arange(len(image_list))
        self.image_list = image_list
        self.classes = classes
        self.input_image_size = (input_image_size)
        self.dataset_size = len(image_list)
        self.coco = COCO(annFile)
        catIds = self.coco.getCatIds(catNms=self.classes)
        self.catIds = catIds
        self.cats = self.coco.loadCats(catIDs)
        self.imgIds = self.coco.getImgIds()
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(len(self.image_list)/self.batch_size)

    def on_epoch_end(self):
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def getClassName(self, classID, cats):
        for i in range(len(cats)):
            if cats[i]['id'] == classID:
                return cats[i]['name']
        return None

    def getNormalMask(self, image_id, catIds):
        annIds = self.coco.getAnnIds(image_id, catIds=catIds, iscrowd=None)
        anns = self.coco.loadAnns(annIds)
        cats = self.coco.loadCats(catIds)
        train_mask = np.zeros(self.input_image_size, dtype=np.uint8)
        for a in range(len(anns)):
            className = self.getClassName(anns[a]['category_id'], cats)
            pixel_value = self.classes.index(className)+1
            new_mask = cv2.resize(self.coco.annToMask(
                anns[a])*pixel_value, self.input_image_size)
            train_mask = np.maximum(new_mask, train_mask)
            # train_mask = new_mask / 255.0
        return train_mask

    def getLevelsMask(self, image_id):
        # for each category , we get the x mask and add it to mask list
        res = []
        mask = np.zeros((self.input_image_size))
        for j, categorie in enumerate(self.catIds):
            annIds = coco.getAnnIds(image_id, catIds=categorie, iscrowd=None)
            anns = coco.loadAnns(annIds)
            mask = self.getNormalMask(image_id, categorie)
            res.append(mask)
        return res

    def getImage(self, file_path):
        train_img = cv2.imread(file_path, cv2.IMREAD_ANYDEPTH)
        train_img = cv2.resize(train_img, (self.input_image_size))
        train_img = train_img.astype(np.float32) / 255.
        if (len(train_img.shape) == 3 and train_img.shape[2] == 3):
            return train_img
        else:
            stacked_img = np.stack((train_img,)*3, axis=-1)
            return stacked_img

    def get_image_Infos_by_path_id(self, node):
        for dict in self.image_list:
            if dict['file_name'] == node:
                return dict

    def __getitem__(self, index):
        X = np.empty((self.batch_size, 128, 128, 3))
        y = np.empty((self.batch_size, 128, 128, 3))
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        for i in range(len(indexes)):
            value = indexes[i]
            img_info = self.image_list[value]
            w = img_info['height']
            h = img_info['width']
            X[i, ] = self.getImage(img_info['file_name'])
            mask_train = self.getLevelsMask(img_info['id'])
            for j in self.catIds:
                y[i, :, :, j] = mask_train[j]
                y[i, :, :, j] = mask_train[j]
                y[i, :, :, j] = mask_train[j]

        X = np.array(X)
        y = np.array(y)

        if self.subset == 'train':
            return X, y
        else:
            return X

In [ ]:
test_generator_class = DataGeneratorFromCocoJson(
    batch_size, "train", images_test, classes, input_image_size, test_path_json, shuffle=False)

In [ ]:
img_s, mask_s = test_generator_class.__getitem__(400)
preds = model.predict(img_s)

In [ ]:
fig = plt.figure(figsize=(10, 25))
gs = gridspec.GridSpec(nrows=len(img_s), ncols=3)
colors = ['yellow', 'green', 'red']
labels = ["Small Bowel", "Large Bowel", "Stomach"]
patches = [mpatches.Patch(
    color=colors[i], label=f"{labels[i]}") for i in range(len(labels))]

cmap1 = mpl.colors.ListedColormap(colors[0])
cmap2 = mpl.colors.ListedColormap(colors[1])
cmap3 = mpl.colors.ListedColormap(colors[2])
flag = False
for i in range(0, 7):

    images, mask = img_s[i], mask_s[i]
    sample_img = images/255.
    mask1 = mask[:, :, 0]
    mask2 = mask[:, :, 1]
    mask3 = mask[:, :, 2]

    pre = preds[i]
    predict1 = pre[:, :, 0]
    predict1 = (predict1 > 0.8).astype(np.float32)
    predict1 = np.array(predict1)
    predict2 = pre[:, :, 1]
    predict3 = pre[:, :, 2]

    ax0 = fig.add_subplot(gs[i, 0])
    im = ax0.imshow(sample_img[:, :, 0], cmap='gray')

    ax1 = fig.add_subplot(gs[i, 1])
    ax2 = fig.add_subplot(gs[i, 2])
    if(flag == False):
        flag = True
        ax0.set_title("Image", fontsize=15, weight='bold', y=1.02)
        ax1.set_title("Mask", fontsize=15, weight='bold', y=1.02)
        ax2.set_title("predicted Mask", fontsize=15, weight='bold', y=1.02)
        plt.legend(handles=patches, bbox_to_anchor=(1.1, 0.65), loc=2, borderaxespad=0.4, fontsize=14,
                   title='Mask Labels', title_fontsize=14, edgecolor="black",  facecolor='#c5c6c7')

    l0 = ax1.imshow(sample_img[:, :, 0], cmap='gray')
    l1 = ax1.imshow(np.ma.masked_where(
        mask1 == False,  mask1), cmap=cmap1, alpha=1)
    l2 = ax1.imshow(np.ma.masked_where(
        mask2 == False,  mask2), cmap=cmap2, alpha=1)
    l3 = ax1.imshow(np.ma.masked_where(
        mask3 == False,  mask3), cmap=cmap3, alpha=1)

    l0 = ax2.imshow(sample_img[:, :, 0], cmap='gray')
    l1 = ax2.imshow(np.ma.masked_where(
        predict1 == False,  predict1), cmap=cmap1, alpha=1)
    l2 = ax2.imshow(np.ma.masked_where(
        predict2 == False,  predict2), cmap=cmap2, alpha=1)
    l3 = ax2.imshow(np.ma.masked_where(
        predict3 == False,  predict3), cmap=cmap3, alpha=1)
    _ = [ax.set_axis_off() for ax in [ax0, ax1]]

    colors = [im.cmap(im.norm(1)) for im in [l1, l2, l3]]

In [ ]:
# Run-length encoding
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]

    return ' '.join(str(x) for x in runs)

In [ ]:
submission_df = pd.DataFrame.from_records(images_test)
submission_df.insert(5, 'prediction', np.nan)
submission_df.insert(3, 'class_name', np.nan)
submission_df.head()

In [ ]:
submission_df.shape

In [ ]:
import gc
test_generator_class = DataGeneratorFromCocoJson(
    1, "test", images_test, classes, input_image_size, test_path_json, shuffle = True)
gc.collect()
LOGITS = model.predict(test_generator_class, verbose=1)
gc.collect()

In [ ]:
lbs = []
sbs = []
sts = []
for index, row in tqdm(submission_df.iterrows(), total=submission_df.shape[0]):
    root_shape = (submission_df.iloc[index]["height"],
                  submission_df.iloc[index]["width"])
    pred_arr = np.round(cv2.resize(
        LOGITS[index, :, :, 0], root_shape, interpolation=cv2.INTER_NEAREST)).astype('uint8')
    sbs.append(rle_encode(pred_arr))
    pred_arr = np.round(cv2.resize(
        LOGITS[index, :, :, 1], root_shape, interpolation=cv2.INTER_NEAREST)).astype('uint8')
    lbs.append(rle_encode(pred_arr))
    pred_arr = np.round(cv2.resize(
        LOGITS[index, :, :, 2], root_shape, interpolation=cv2.INTER_NEAREST)).astype('uint8')
    sts.append(rle_encode(pred_arr))
del LOGITS
gc.collect()    

In [ ]:
ids = []
classes = []
rles = []
for index, row in tqdm(submission_df.iterrows(), total=submission_df.shape[0]):
    ids.extend([row['file_id']] * 3)
    classes.extend(['small_bowel', 'large_bowel', 'stomach'])
    rles.extend([sbs[index], lbs[index], sts[index]])

In [ ]:
submission_df = pd.DataFrame()
submission_df['id'] = ids
submission_df['class'] = classes
submission_df['predicted'] = rles
submission_df.to_csv("submission.csv", index=False)

In [ ]:
submission_df.sort_values('id')